In [2]:
import re
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import datetime as dt
import sqlite3

In [3]:
'''scrapes game url returning a dict'''
url = 'https://www.pro-football-reference.com/boxscores/202009100kan.htm'
res = requests.get(url)
# use re to escape html comments
comm = re.compile("<!--|-->")
soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
# get content div, holds all body content
divs = soup.findAll('div', id = "content")
# table for points by quarter
points = pd.read_html(res.text)[0]
# table for scoring stats
stats = pd.read_html(str(divs[0].findAll(
    "div",id=re.compile("^all_team_stats"))[0].findAll('table')[0]))[0]

# get datetime object for the dict
date_ = divs[0].findAll("div", class_='scorebox')[0].findAll(
    'div',class_='scorebox_meta')[0].findAll('div')[0].text
time_ = divs[0].findAll("div", class_='scorebox')[0].findAll(
    'div', class_='scorebox_meta')[0].findAll('div')[1].text.replace('Start Time: ', '')
datestr = date_ + ' ' + time_
dateobj = dt.datetime.strptime(datestr, '%A %b %d, %Y %I:%M%p')

# get week number, all of the playoffs is called week 18
if soup.findAll('h2')[0].text.endswith('Playoffs') == True:
    week_ = '18'
else:
    week_ = soup.findAll('h2')[0].text.replace('NFL Scores — Week ', '')

if dateobj.month >= 7:
    season_ = dateobj.year
else:
    season_ = dateobj.year - 1

game_dict = {}

In [4]:
pd.read_html(res.text)[0]

,Unnamed: 0,Unnamed: 1,1,2,3,4,Final
0,via Sports Logos.net About logos,Houston Texans,7,0,0,13,20
1,via Sports Logos.net About logos,Kansas City Chiefs,0,17,7,10,34


In [5]:
stats.iloc[5,1]

'360'

In [6]:
stats

,Unnamed: 0,HOU,KAN
0,First Downs,21,28
1,Rush-Yds-TDs,22-118-2,34-166-1
2,Cmp-Att-Yd-TD-INT,20-32-253-1-1,24-32-211-3-0
3,Sacked-Yards,4-11,1-8
4,Net Pass Yards,242,203
5,Total Yards,360,369
6,Fumbles-Lost,0-0,0-0
7,Turnovers,1,0
8,Penalties-Yards,5-37,1-5
9,Third Down Conv.,4-10,7-13


In [7]:
stats.iloc[3, 2].split('-')[0]

'1'

In [19]:
h_drives = pd.read_html(str(divs[0].findAll(
    "table",id='home_drives')))[0]

In [22]:
len(pd.read_html(str(divs[0].findAll(
    "table",id='home_drives')))[0].index)

9

In [20]:
len(h_drives.index)

9

In [32]:
divs[0].findAll(
    "table",id='home_drives')

[<table class="sortable stats_table" data-cols-to-freeze=",1" id="home_drives">
 <caption>Chiefs Drives Table</caption>
 <colgroup><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
 <thead>
 <tr>
 <th aria-label="#" class="poptip center" data-stat="drive_num" scope="col">#</th>
 <th aria-label="Quarter" class="poptip center" data-stat="quarter" scope="col">Quarter</th>
 <th aria-label="Time" class="poptip center" data-stat="time_start" scope="col">Time</th>
 <th aria-label="LOS" class="poptip center" data-stat="start_at" scope="col">LOS</th>
 <th aria-label="Number of plays in drive.P - Pass, R - Rush, Y - Penalty" class="poptip center" data-stat="play_count_tip" data-tip="Number of plays in drive.&lt;br /&gt;P - Pass, R - Rush, Y - Penalty" scope="col">Plays</th>
 <th aria-label="Length" class="poptip center" data-stat="time_total" scope="col">Length</th>
 <th aria-label="Net Yards (Just Plays)" class="poptip center" data-stat="net_yds" scope="col">Net_yds</th>
 <th aria-lab